# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [85]:
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from api_keys import g_key


Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [86]:
#CityData.csv
vacation_data = pd.read_csv('../WeatherPy/CityData.csv')
vacation_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jamestown,42.0970,-79.2353,11.08,85,90,4.61,US,1612763527
1,yeppoon,-23.1333,150.7333,91.40,55,75,19.57,AU,1612763738
2,usinsk,65.9939,57.5281,-16.80,87,100,15.70,RU,1612763738
3,innisfail,-17.5333,146.0333,86.61,72,98,6.35,AU,1612763738
4,zhelyabovka,45.3988,34.7578,34.75,94,100,13.15,UA,1612763738


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [87]:
gmaps.configure(api_key=g_key)
locations = vacation_data[['Lat', 'Lng']]
humidity = vacation_data['Humidity']

fig = gmaps.figure(center=(10,-5),zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                               max_intensity=100,point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [88]:
ideal_weather = vacation_data[(vacation_data['Max Temp'] < 80) & 
                              (vacation_data['Max Temp'] > 70) &
                              (vacation_data['Wind Speed'] < 10) &
                              (vacation_data['Cloudiness'] == 0) ]
ideal_weather.head()                            

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
209,ormara,25.2088,64.6357,73.44,22,0,2.62,PK,1612763772
232,raha,26.2333,92.5167,76.60,36,0,1.32,IN,1612763776
239,lima,-12.0432,-77.0282,71.28,88,0,9.22,PE,1612763718
249,ambah,26.7075,78.2261,73.87,29,0,5.55,IN,1612763778
344,tura,25.5198,90.2201,79.93,23,0,5.19,IN,1612763794


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [89]:
hotel_df = ideal_weather[['Lat','Lng','City','Country']].copy().reset_index().drop(columns='index')
hotel_df['Hotel Name'] = ''
hotel_df

,Lat,Lng,City,Country,Hotel Name
0,25.2088,64.6357,ormara,PK,
1,26.2333,92.5167,raha,IN,
2,-12.0432,-77.0282,lima,PE,
3,26.7075,78.2261,ambah,IN,
4,25.5198,90.2201,tura,IN,
5,26.8516,89.3884,phuntsholing,BT,
6,10.9667,-3.2500,diebougou,BF,
7,15.7432,-96.4661,pochutla,MX,
8,10.3997,-75.5144,cartagena,CO,
9,20.8667,-90.4000,celestun,MX,


In [116]:
for index, row in hotel_df.iterrows():
    try:
        url ='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
        params = {
         'radius': 5000,
         'keyword': 'hotel',
         'key': g_key }
        latitude = row['Lat']
        longitude = row['Lng']
        params['location'] = f'{latitude},{longitude}'
        res = requests.get(url,params=params).json()
        hotel_df.loc[index, 'Hotel Name'] = res['results'][0]['name']
        if res['results'][0]['name'] == '':
            hotel_df.loc[index, 'Hotel Name'] = 'Na'
        
    except:
        pass
hotel_df

,Lat,Lng,City,Country,Hotel Name
0,25.2088,64.6357,ormara,PK,Alqadri hotel zero point ormara
1,26.2333,92.5167,raha,IN,
2,-12.0432,-77.0282,lima,PE,Hotel Carrera
3,26.7075,78.2261,ambah,IN,Rathore Hotel
4,25.5198,90.2201,tura,IN,Hotel Polo Orchid
5,26.8516,89.3884,phuntsholing,BT,Hotel Bhutan Ga Me Ga
6,10.9667,-3.2500,diebougou,BF,HÔTEL DEPAAKOU
7,15.7432,-96.4661,pochutla,MX,Hotel Posada San Jose
8,10.3997,-75.5144,cartagena,CO,Bastión Luxury Hotel
9,20.8667,-90.4000,celestun,MX,Playa 55 beach escape


In [103]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [104]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotels)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))